In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
!pip install comet_ml
from comet_ml import Experiment
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import joblib
import time
import matplotlib.pyplot as plt

sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [3]:
# Importo los datos
features = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50_PCA95.joblib"))
label = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib"))

%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"
# Importo particiones
indices_train = np.array(joblib.load('indices_train_W200_I50_sujeto.pkl'))
indices_val = np.array(joblib.load('indices_val_W200_I50_sujeto.pkl'))
indices_test = np.array(joblib.load('indices_test_W200_I50_sujeto.pkl'))
sorteo = np.array(joblib.load('sorteo_sujetos.pkl'))

/content/drive/My Drive/repo_tesis/archivos_generados_codigos


Entreno el modelo

In [5]:
start_time = time.time()

inputDim = features.shape[1]  # Esto lo cambié. Me fijo las dimensiones directamente en las dimensiones de la matriz
clasNum = np.max(label[:, 1]) + 1

# Borrar todo lo que no use de aqui para adelante en este punto

# Architecture 250 - 500 - 250
red = cl.Clasificador(model= "mlp", arch = [250,500,250], catNum = clasNum, featureNum = inputDim)
red.train(features[indices_train, :], label[indices_train, 1], features[indices_val, :], label[indices_val, 1])

end_time = time.time()
training_time = end_time - start_time
y_pred = red.predict(features[indices_test, :])

nombre = 'ronda1_exp_mlp_sep_sub_pca95_n1'

joblib.dump(red, nombre + '.pkl')
y_pred = red.predict(features[indices_test, :])
accuracy = accuracy_score(label[indices_test, 1], y_pred)
precision = precision_score(label[indices_test, 1], y_pred, average='weighted')
recall = recall_score(label[indices_test, 1], y_pred, average='weighted')
conf_matrix = confusion_matrix(label[indices_test, 1], y_pred)
# plt.savefig("confusion_matrix.png")

# Conectar con Comet
API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO' # Se puede encontrar en Settings(Arriba a la derecha en Comet)
exp = Experiment(api_key=API_KEY,
              project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
              auto_param_logging=False)
exp.set_name(nombre) # Nombre de este experimento
exp.add_tags(['exp1', 'mlp', 'sep_sub', 'pca95']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(label[indices_test, 1], y_pred)
exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
exp.log_text("Se fijan 2 sujetos de test y se sortea el resto. El primer y segundo sujeto del sorteo son los de validación.")   # Comentario del experimento   # Comentario del experimento
exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
exp.end()

Epoch 1/50
2790/2790 [==============================] - 12s 4ms/step - loss: 1.1429 - accuracy: 0.6108 - val_loss: 2.4430 - val_accuracy: 0.3234
Epoch 2/50
2790/2790 [==============================] - 12s 4ms/step - loss: 0.7425 - accuracy: 0.7418 - val_loss: 3.0167 - val_accuracy: 0.3221
Epoch 3/50
2790/2790 [==============================] - 12s 4ms/step - loss: 0.5493 - accuracy: 0.8087 - val_loss: 3.5098 - val_accuracy: 0.3286
Epoch 4/50
2790/2790 [==============================] - 12s 4ms/step - loss: 0.4242 - accuracy: 0.8512 - val_loss: 4.1850 - val_accuracy: 0.3071
Epoch 5/50
2790/2790 [==============================] - 12s 4ms/step - loss: 0.3290 - accuracy: 0.8849 - val_loss: 4.6473 - val_accuracy: 0.3198
Epoch 6/50
2790/2790 [==============================] - 11s 4ms/step - loss: 0.2606 - accuracy: 0.9111 - val_loss: 5.3553 - val_accuracy: 0.3028
Epoch 7/50
2790/2790 [==============================] - 12s 4ms/step - loss: 0.2160 - accuracy: 0.9260 - val_loss: 6.0844 - val_ac

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : ronda1_exp_mlp_sep_sub_pca95_n1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/a4e50441ddca4bf3be54e6bd551743e9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [51]                  : (0.6052248580992577, 0.9869605898857117)
COMET INFO:     batch_accuracy [13950]         : (0.0625, 1.0)
COMET INFO:     batch_loss [13950]             : (5.818683803227032e-06, 2.5992274284362793)
COMET INFO:     epoch_duration [50]            : (10.028066917999922, 12.900623000999985)
COMET INFO:     loss [50]                      : (0.052998967468738556, 1.1429475545883179)
COMET INFO:     precision